In [ ]:
#these commands can be run on the shell and get the data with the command wget
#the cell needs to be run only once
# The file for 1993 does not exist
!for ((y=93; y<=99; y+=1)); do wget \
ftp://ftp.census.gov/Econ2001_And_Earlier/CBP_CSV/zbp$y\totals.zip; done

!for ((y=0; y<=1; y+=1)); do wget \
ftp://ftp.census.gov/Econ2001_And_Earlier/CBP_CSV/zbp0$y\totals.zip; done

!for ((y=2; y<=9; y+=1)); do wget \
ftp://ftp.census.gov/econ200$y\/CBP_CSV/zbp0$y\totals.zip; done

!for ((y=10; y<=14; y+=1)); do wget \
ftp://ftp.census.gov/econ20$y\/CBP_CSV/zbp$y\totals.zip; done

In [5]:
! wget https://data.cityofnewyork.us/download/i8iw-xf4u/application%2Fzip > ZIP_CODE_040114.zip

--2017-12-20 01:52:46--  https://data.cityofnewyork.us/download/i8iw-xf4u/application%2Fzip
Resolving data.cityofnewyork.us... 52.206.140.199
Connecting to data.cityofnewyork.us|52.206.140.199|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.cityofnewyork.us/api/views/i8iw-xf4u/files/YObIR0MbpUVA0EpQzZSq5x55FzKGM2ejSeahdvjqR20?filename=ZIP_CODE_040114.zip [following]
--2017-12-20 01:52:46--  https://data.cityofnewyork.us/api/views/i8iw-xf4u/files/YObIR0MbpUVA0EpQzZSq5x55FzKGM2ejSeahdvjqR20?filename=ZIP_CODE_040114.zip
Connecting to data.cityofnewyork.us|52.206.140.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1514401 (1.4M) [application/octet-stream]
Saving to: “application%2Fzip.1”

100%[======================================>] 1,514,401   1.93M/s   in 0.7s    

2017-12-20 01:52:47 (1.93 MB/s) - “application%2Fzip.1” saved [1514401/1514401]



In [2]:
import pandas as pd
import zipfile 
import os
import geopandas as gpd
#import choroplethNYC as cp
import matplotlib.pylab as pylab
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
# Download the shape file for NYC and put the uncompressed directory in $PUIDATA

In [7]:
nyc_shp = gpd.GeoDataFrame.from_file(os.getenv("PUIDATA") + "/ZIP_CODES/" + 
                                     "ZIP_CODE_040114.shp")

In [9]:
# Check to make sure plotting works
#cp.choroplethNYC(nyc_shp, column=None, color="white", edgecolor="black", lw=2)

In [10]:
# Handling the .zip files
year_list = ['94', '95', '96', '97', '98', '99', '00', '01', '02',
             '03', '04', '05', '06', '07', '08', '09', '10', '11',
             '12', '13', '14']
full_year_list = ['1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002',
             '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011',
             '2012', '2013', '2014']
d={}
for i in range(len(year_list)):
    fname = 'zbp'+ str(year_list[i]) +'totals.zip'
    zf = zipfile.ZipFile(fname)
    df = pd.read_csv(zf.open(fname.replace('.zip','.txt')))
    df = df[df.iloc[:,0].isin(nyc_shp['ZIPCODE'])]
    df = df.rename(columns={'EST': 'est', 'ZIP': 'zip'})
    df['year'] = full_year_list[i]
    d[year_list[i]] = df[['zip', 'est', 'year']]

# Get rid of the non shared zipcodes
zip_list = list(d['94']['zip'])
print(len(zip_list))
for i in d:
    zip_list = list(set(zip_list) & set(d[i]['zip']))
print(len(zip_list))

for i in d:
    d[i] = d[i][d[i]['zip'].isin(zip_list)]
    print(d[i].shape)

FileNotFoundError: [Errno 2] No such file or directory: 'zbp94totals.zip'

In [11]:
# Just going to write this out...
all_df = pd.DataFrame({'zip': d['94']['zip'].values, 
                      '1994': d['94']['est'].values,
                      '1995': d['95']['est'].values,
                      '1996': d['96']['est'].values,
                      '1997': d['97']['est'].values,
                      '1998': d['98']['est'].values,
                      '1999': d['99']['est'].values,
                      '2000': d['00']['est'].values,
                      '2001': d['01']['est'].values,
                      '2002': d['02']['est'].values,
                      '2003': d['03']['est'].values,
                      '2004': d['04']['est'].values,
                      '2005': d['05']['est'].values,
                      '2006': d['06']['est'].values,
                      '2007': d['07']['est'].values,
                      '2008': d['08']['est'].values,
                      '2009': d['09']['est'].values,
                      '2010': d['10']['est'].values,
                      '2011': d['11']['est'].values,
                      '2012': d['12']['est'].values,
                      '2013': d['13']['est'].values,
                      '2014': d['14']['est'].values})

KeyError: '94'

In [12]:
all_df = all_df.set_index('zip')
all_df.head()

NameError: name 'all_df' is not defined

In [13]:
# Now whiten the data
from sklearn import preprocessing
for col in all_df:
    all_df[col] = preprocessing.scale(all_df[col])
all_df.describe()

NameError: name 'all_df' is not defined

For loop with scikitlearn made all means 0 and std devs 1.

In [ ]:
from sklearn import cluster
clusters = cluster.KMeans(n_clusters=4, random_state=39).fit(all_df)

In [ ]:
#for i in range
clusters.cluster_centers_
clusters.labels_

In [ ]:
plt.figure(figsize=(15,15))
ticks = range(1994, 2015)

plt.subplot(2, 2, 1)
for i in range(len(clusters.labels_)):
    if clusters.labels_[i] == 0:
        plt.plot(range(all_df.shape[1]), all_df.iloc[i,:], color = 'blue', alpha =0.4)
plt.plot(range(all_df.shape[1]), clusters.cluster_centers_[0], color='black',  linewidth=5.0)
plt.xlabel('Year')
plt.ylabel('Establishments')
plt.xticks(range(20), ticks, rotation='vertical')
plt.title('Cluster 1')

plt.subplot(2, 2, 2)
for i in range(len(clusters.labels_)):
    if clusters.labels_[i] == 1:
        plt.plot(range(all_df.shape[1]), all_df.iloc[i,:], color = 'blue', alpha =0.4)
plt.plot(range(all_df.shape[1]), clusters.cluster_centers_[1], color='black', linewidth=5.0)
plt.xlabel('Year')
plt.ylabel('Establishments')
plt.xticks(range(20), ticks, rotation='vertical')
plt.title('Cluster 2')

plt.subplot(2, 2, 3)
for i in range(len(clusters.labels_)):
    if clusters.labels_[i] == 2:
        plt.plot(range(all_df.shape[1]), all_df.iloc[i,:], color = 'blue', alpha =0.4)
plt.plot(range(all_df.shape[1]), clusters.cluster_centers_[2], color='black', linewidth=5.0)
plt.xlabel('Year')
plt.ylabel('Establishments')
plt.xticks(range(20), ticks, rotation='vertical')
plt.title('Cluster 3')

plt.subplot(2, 2, 4)
for i in range(len(clusters.labels_)):
    if clusters.labels_[i] == 3:
        plt.plot(range(all_df.shape[1]), all_df.iloc[i,:], color = 'blue', alpha =0.4)
plt.plot(range(all_df.shape[1]), clusters.cluster_centers_[3], color='black', linewidth=5.0)
plt.xlabel('Year')
plt.ylabel('Establishments')
plt.xticks(range(20), ticks, rotation='vertical')
plt.title('Cluster 4')

Figure 1:
Split into 4 plots are the 4 different kmeans clusters of ZIP codes with the whitened values for establishments on the y axis and the time on the x-axis.

In [ ]:
# make chloroplaeth
all_df['label'] = clusters.labels_
all_df['ZIP'] = all_df.index
all_df['ZIP'] = all_df['ZIP'].astype(str)
nyc_shp['ZIPCODE'] = nyc_shp['ZIPCODE'].astype(str)
merged_df = pd.merge(nyc_shp, all_df, left_on='ZIPCODE', right_on='ZIP')
merged_df.head()

In [ ]:
cp.choroplethNYC(merged_df, column='label', color="white", edgecolor="black", lw=2)


Figure 2:
Chloropleth map displaying the 4 cluster k-mean cluster of the number of establishment over 1994-2014


In [ ]:
# Now will use hierarchical  instead
spec = cluster.SpectralClustering(n_clusters=4).fit(all_df)

In [ ]:
spec.labels_

In [ ]:
# make chloroplaeth for spectral clusters
all_df['label'] = spec.labels_
all_df['ZIP'] = all_df.index
all_df['ZIP'] = all_df['ZIP'].astype(str)
nyc_shp['ZIPCODE'] = nyc_shp['ZIPCODE'].astype(str)
merged_df = pd.merge(nyc_shp, all_df, left_on='ZIPCODE', right_on='ZIP')
merged_df.head()

In [ ]:
cp.choroplethNYC(merged_df, column='label', color="white", edgecolor="black", lw=2)


Figure 3:
Chloropleth map displaying the 4 cluster spectral clustering of the number of establishment over 1994-2014


From the 2 different types of clustering I have done here the main event that stand out to me over the period of 1994 to 2014 is the clustering of ZIP codes in outer Queens having a declining number of establishments.  Many of these ZIP codes are in the same clusters with each other for both the k-means and spectral clustering.  When plotting the center of this cluster for k-mean (Cluster 4), there is a a steady decline.

Points - 8 - data not reproducible